# Downloading Planet Datasets

This tutorial uses the Planet API to find and download planetscope data.

To have access to the Planet API to run this tutorial (https://www.planet.com)


## Differences between notebooks and scripts

Please note that there are some differences between using the functions provided in RSGISLib when running from within a notebook, such as this, verses a normal python scripts as the python API uses asyncio (https://docs.python.org/3/library/asyncio.html) as does the notebook server and two loops cannot be run alongside one other.

Therefore, I have shown both functions and prepended the cells with functions that would be used within a normal Python script with %%script false --no-raise-error so they do not run within the notebook and produce an error. Note the function names are almost the same with the functions executed within the notebook starting with an `_` and those used within a script with `run_`


## Import Modules

In [ ]:
import datetime
import os

import rsgislib.tools.utils
from rsgislib.dataaccess import planet_data_api

## Create Output Directory

In [ ]:
out_dir_path = "planet_data"
if not os.path.exists(out_dir_path):
    os.mkdir(out_dir_path)

## Username and Password

You need to be careful with your username and password and therefore you should not write them into the notebook. RSGISlib provide a tool/functions for doing a basic encoding of the username and password so they are not stored as free text (Note. be careful as the simple encoding is not secure). 

To create the encoded file, you can use the command line tool `rsgisuserpassfile.py` as shown below:

`rsgisuserpassfile.py planet_userinfo.txt`

Once you have created the `planet_userinfo.txt` file, you can read it into your notebook/script using the `get_username_password` function shown below:

In [ ]:
username, password = rsgislib.tools.utils.get_username_password(
    input_file="planet_userinfo.txt"
)

## 1. Get Authentication Object

When you use the API, you need to be authenticated. The function below will create an authentication object using your login credencials.

In [ ]:
planet_auth = planet_data_api.planet_auth(username=username, password=password)

## 2. Search for Scenes

The first step is to query to find the scenes you are interested in. In this case, we are looking for scenes in May and June 2024 within a bounding box (xMin, xMax, yMin, yMax) and with cloud cover of less than 10 percent. 

The results have been restricted to just provide 2 scenes to speed up building the order and the download. Given the small land area of the scenes you would probably increase this. 

### Notebook Function:

In [ ]:
items = await planet_data_api._search_planet_items(
    planet_auth,
    item_type=planet_data_api.RSGIS_PLANET_ITEM_PSScene,
    bbox=(-4.1, -4.0, 52.4, 52.5),
    start_date=datetime.datetime(year=2024, month=5, day=1),
    end_date=datetime.datetime(year=2024, month=6, day=30),
    cloud_cover=10,
    sun_elevation_min=None,
    sun_elevation_max=None,
    view_angle_min=None,
    view_angle_max=None,
    max_n_rslts=2,
)

### Script Function

In [ ]:
%%script false --no-raise-error

items = planet_data_api.run_search_planet_items(
    planet_auth,
    item_type = planet_data_api.RSGIS_PLANET_ITEM_PSScene,
    bbox = (-4.1, -4.0, 52.4, 52.5),
    start_date = datetime.datetime(year=2024, month=5, day=1),
    end_date = datetime.datetime(year=2024, month=6, day=30),
    cloud_cover = 10,
    sun_elevation_min = None,
    sun_elevation_max = None,
    view_angle_min = None,
    view_angle_max = None,
    max_n_rslts = 2,
)

In [ ]:
items

## 3. Create an Order

The next step is to create an order so the planet servers will prepare the data to be downloaded. You need to provide the list of items you found through the search, specify the type of the items (which will match what you specified in the search), and then the bundle you want to download (e.g., 8-band surface reflectance). 

By default, once the order has been created and is ready to download, Planet will send you an email notification. Although this can be turned off. 

### Notebook Function:

In [ ]:
order_info = await planet_data_api._create_planet_order(
    planet_auth,
    order_name="example_order",
    items=items,
    item_type=planet_data_api.RSGIS_PLANET_ITEM_PSScene,
    bundle_type=planet_data_api.RSGIS_PLANET_BUNDLE_analytic_8b_sr_udm2,
    email_notification=True,
)

### Script Function:

In [ ]:
%%script false --no-raise-error

order_info = planet_data_api.run_create_planet_order(
        planet_auth,
        order_name = "example_order",
        items = items,
        item_type = planet_data_api.RSGIS_PLANET_ITEM_PSScene,
        bundle_type = planet_data_api.RSGIS_PLANET_BUNDLE_analytic_8b_sr_udm2,
        email_notification = True,
)

Note. you will want to make note of order id - the `id` field in the returned order_info dictionary.

In [ ]:
order_info

## 4. Download the Order

Once the order has been submitted, you should able to see it on the Planet orders page, as shown below:

![Planet Order](figures/planet_orders_page.png)

Once the status has changed to `success` the data is available for download and you can run the following function - note, if you run the function before the data is ready you will get an error. Note that it can take a little while for the data to be available for download, so you might need some patience. 

### Notebook Function:

In [ ]:
dwnld_paths = await planet_data_api._download_planet_order(
    planet_auth,
    order_id=order_info["id"],
    out_file_path=out_dir_path,
    overwrite=False,
)

### Script Function:

In [ ]:
%%script false --no-raise-error

dwnld_paths = planet_data_api.run_download_planet_order(
    planet_auth,
    order_id = order_info["id"],
    out_file_path = out_dir_path,
    overwrite = False,
)

The download function will return a list of the file paths which have been downloaded:

In [ ]:
dwnld_paths